## Assignment 4

### Question 3

#### Create programmatic AWS Lambda Function and request resources

In [1]:
import boto3
import json

# Access IAM role,to allow Lambda to interact with other AWS resources
aws_lambda = boto3.client('lambda')
iam_client = boto3.client('iam')
role = iam_client.get_role(RoleName='LabRole')

# Open zipped directory that contains lambda function
with open('q2.zip', 'rb') as f:
    lambda_zip = f.read() # Read Python function

try:
    # If function hasn't yet been created, create it
    response = aws_lambda.create_function(
        FunctionName='q2_programmatic',
        Runtime='python3.9',
        Role=role['Role']['Arn'], 
        Handler='lambda_function.lambda_handler', 
        Code=dict(ZipFile=lambda_zip),
        Timeout=300
    )
except aws_lambda.exceptions.ResourceConflictException:
    # If function already exists, update it based on zip file contents
    response = aws_lambda.update_function_code(
        FunctionName='q2_programmatic',
        ZipFile=lambda_zip
        )

lambda_arn = response['FunctionArn']

#### Invoke Lambda function with three test data cases

In [2]:
# (1) Success case test data where the entry is valid

test_data_1 = {
    "user_id": "0001",
    "timestamp": "092821120000",
    "time_elapsed": 5,
    "q1": 5,
    "q2": 3,
    "q3": 2,
    "q4": 2,
    "q5": 4,
    "freetext": "I had a very bad day today..."
}

# Run synchronously and print response:
r = aws_lambda.invoke(FunctionName='q2_programmatic',
                      InvocationType='RequestResponse',
                      Payload=json.dumps(test_data_1))
json.loads(r['Payload'].read())

{'statusCode': 200, 'body': 'OK'}

In [3]:
# (2) Invalid Entry: Not enough time elapsed

test_data_2 = {
    "user_id": "0001",
    "timestamp": "092821120001",
    "time_elapsed": 2,
    "q1": 5,
    "q2": 3,
    "q3": 2,
    "q4": 2,
    "q5": 4,
    "freetext": "I had a very bad day today..."
 }

# Run synchronously and print response:
r = aws_lambda.invoke(FunctionName='q2_programmatic',
                      InvocationType='RequestResponse',
                      Payload=json.dumps(test_data_2))
json.loads(r['Payload'].read())

{'statusCode': 400, 'body': 'Invalid entry. Not enough time elapsed.'}

In [4]:
# (3) Invalid Entry: No text response

test_data_3 = {
    "user_id": "0001",
    "timestamp": "092821120100",
    "time_elapsed": 5,
    "q1": 5,
    "q2": 3,
    "q3": 2,
    "q4": 2,
    "q5": 4,
    "freetext": ""
}

# Run synchronously and print response:
r = aws_lambda.invoke(FunctionName='q2_programmatic',
                      InvocationType='RequestResponse',
                      Payload=json.dumps(test_data_3))
json.loads(r['Payload'].read())

{'statusCode': 400, 'body': 'Invalid entry. No text response'}